# **Super Bowl LX Chatbot** 🏈

## ✨ Features:
- **Groq LLM**
- **Accurate answers** from actual game data
- **Proper citations** with working links
- **Hybrid retrieval** for best results


In [28]:
# Installation
!pip -q install sentence-transformers faiss-cpu rank-bm25 groq
!pip -q install beautifulsoup4 requests gradio

In [29]:
import os
from getpass import getpass

# Get FREE Groq API key from: https://console.groq.com/keys
if 'GROQ_API_KEY' not in os.environ:
    print("🔑 Get your FREE Groq API key from: https://console.groq.com/keys")
    print("   (Sign up with Google/GitHub - takes 30 seconds!)\n")
    os.environ['GROQ_API_KEY'] = getpass('Enter your Groq API key: ')

from groq import Groq
client = Groq(api_key=os.environ['GROQ_API_KEY'])

print("✅ Groq configured! Using Llama 3.3 70B model")

✅ Groq configured! Using Llama 3.3 70B model


## 1. Game Data & Document Loading

In [30]:
import requests
from bs4 import BeautifulSoup
from typing import List, Dict, Tuple
import re

# ACTUAL SUPER BOWL LX DATA (February 8, 2026)
GAME_FACTS = """
SUPER BOWL LX - COMPLETE GAME SUMMARY
Date: February 8, 2026
Teams: Seattle Seahawks vs New England Patriots
Final Score: Seahawks 29, Patriots 13
Location: Gillette Stadium, Foxborough, MA

GAME MVP: Information not yet available in official sources.
Top Performers:
- Jason Myers (SEA kicker): 5/5 field goals (100%), 17 points scored
- Uchenna Nwosu (SEA linebacker): Pick-six TD, game-sealing play
- Seattle Defense: 6 sacks, 2 INT, dominated Patriots offense

QUARTERBACKS:
- Seattle: Sam Darnold - 19/38, 202 yards, 1 TD, 0 INT
- New England: Drake Maye - 27/43, 295 yards, 2 TD, 2 INT, 6 sacks taken

SCORING SUMMARY:
Q1: SEA 3-0 (Myers 33-yard FG)
Q2: SEA 9-0 (Myers 39-yard FG, Myers 41-yard FG)
Q3: SEA 12-0 (Myers 41-yard FG)
Q4: SEA 29-13
  - Darnold 16-yd TD to Barner (SEA 19-0)
  - Maye 35-yd TD to Hollins (SEA 19-7)
  - Myers 26-yard FG (SEA 22-7)
  - Nwosu 45-yd INT return TD (SEA 29-7)
  - Maye 7-yd TD to Stevenson (SEA 29-13, 2pt failed)

KEY STATISTICS:
Total Yards: SEA 335, NE 331
Time of Possession: SEA 33:11, NE 26:49
Turnovers: SEA 0, NE 3 (2 INT, 1 fumble)
Sacks: SEA 6, NE 1
First Downs: SEA 20, NE 18

DEFENSE:
Seahawks: 6 sacks (43 yards), 2 INT (80 return yards, 1 TD), 1 fumble recovery
Patriots: 1 sack (8 yards), 0 INT

SPECIAL TEAMS:
Jason Myers (SEA): 5/5 FG, 4/4 XP (kicks: 26, 33, 39, 41, 41 yards)
Alex Borregales (NE): 0/0 FG, 2/3 XP

GAME NOTES:
- Patriots were shut out for 3 quarters (0-12)
- All Patriots scoring came in 4th quarter
- Seahawks dominated time of possession
- Drake Maye sacked 6 times, hurried throughout
- Uchenna Nwosu's pick-six sealed the victory at 4:37 Q4
"""

# Add your article URLs here
URLS = [
    # Add Super Bowl LX coverage URLs
    # "https://www.espn.com/nfl/...",
    # "https://www.nfl.com/...",
]

def fetch_url_content(url: str) -> Tuple[str, str]:
    """Fetch content and return (text, url)"""
    try:
        headers = {'User-Agent': 'Mozilla/5.0'}
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.content, 'html.parser')

        for script in soup(["script", "style", "nav", "footer"]):
            script.decompose()

        text = soup.get_text()
        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk)

        return text, url
    except Exception as e:
        print(f"⚠️ Error fetching {url}: {e}")
        return "", url

def smart_chunking(text: str, chunk_size: int = 400, overlap: int = 100) -> List[str]:
    """Sentence-based chunking with overlap"""
    sentences = re.split(r'(?<=[.!?])\s+', text)

    chunks = []
    current_chunk = []
    current_size = 0

    for sent in sentences:
        sent_len = len(sent.split())

        if current_size + sent_len > chunk_size and current_chunk:
            chunks.append(' '.join(current_chunk))

            # Keep last sentences for overlap
            overlap_sents = []
            overlap_size = 0
            for s in reversed(current_chunk):
                s_len = len(s.split())
                if overlap_size + s_len <= overlap:
                    overlap_sents.insert(0, s)
                    overlap_size += s_len
                else:
                    break

            current_chunk = overlap_sents
            current_size = overlap_size

        current_chunk.append(sent)
        current_size += sent_len

    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks

# Load documents
print("📚 Loading documents...\n")
all_chunks = []
sources = []

# Add official game facts first
fact_chunks = smart_chunking(GAME_FACTS, chunk_size=300, overlap=50)
all_chunks.extend(fact_chunks)
sources.extend(["Official Game Stats"] * len(fact_chunks))
print(f"✅ Loaded {len(fact_chunks)} chunks from official game data")

# Load web articles
if URLS:
    for url in URLS:
        content, source_url = fetch_url_content(url)
        if content:
            chunks = smart_chunking(content, chunk_size=400, overlap=100)
            all_chunks.extend(chunks)
            sources.extend([source_url] * len(chunks))
            print(f"✅ Loaded {len(chunks)} chunks from {url}")
else:
    print("ℹ️ No URLs provided - using official game data only")
    print("   Add URLs to the URLS list for additional coverage")

print(f"\n📊 Total chunks loaded: {len(all_chunks)}")

📚 Loading documents...

✅ Loaded 1 chunks from official game data
ℹ️ No URLs provided - using official game data only
   Add URLs to the URLS list for additional coverage

📊 Total chunks loaded: 1


## 2. Hybrid Retrieval System

In [31]:
from sentence_transformers import SentenceTransformer
from rank_bm25 import BM25Okapi
import numpy as np
import faiss

print("🔧 Building retrieval system...\n")

# Dense embeddings
embedder = SentenceTransformer('all-MiniLM-L6-v2')
print("Creating embeddings...")
embeddings = embedder.encode(all_chunks, show_progress_bar=True)

# FAISS index
dimension = embeddings.shape[1]
index = faiss.IndexFlatIP(dimension)
faiss.normalize_L2(embeddings)
index.add(embeddings)

# BM25 index
tokenized_chunks = [chunk.lower().split() for chunk in all_chunks]
bm25 = BM25Okapi(tokenized_chunks)

print("\n✅ Retrieval system ready!")

def hybrid_retrieve(query: str, k: int = 15, alpha: float = 0.5) -> List[Tuple[str, str, float]]:
    """Hybrid retrieval combining dense + sparse search"""
    # Dense retrieval (FAISS)
    query_vec = embedder.encode([query])
    faiss.normalize_L2(query_vec)
    dense_scores, dense_ids = index.search(query_vec, min(k * 2, len(all_chunks)))
    dense_scores = dense_scores[0]
    dense_ids = dense_ids[0]

    # Sparse retrieval (BM25)
    query_tokens = query.lower().split()
    bm25_scores = bm25.get_scores(query_tokens)

    # Normalize scores
    if dense_scores.max() > dense_scores.min():
        dense_norm = (dense_scores - dense_scores.min()) / (dense_scores.max() - dense_scores.min())
    else:
        dense_norm = dense_scores

    if bm25_scores.max() > bm25_scores.min():
        bm25_norm = (bm25_scores - bm25_scores.min()) / (bm25_scores.max() - bm25_scores.min())
    else:
        bm25_norm = bm25_scores

    # Combine scores
    combined = {}
    for idx, score in zip(dense_ids, dense_norm):
        if idx < len(all_chunks):
            combined[idx] = alpha * score

    for idx, score in enumerate(bm25_norm):
        if idx in combined:
            combined[idx] += (1 - alpha) * score
        else:
            combined[idx] = (1 - alpha) * score

    # Sort and return
    ranked = sorted(combined.items(), key=lambda x: x[1], reverse=True)[:k]
    results = [(all_chunks[idx], sources[idx], score) for idx, score in ranked]

    return results

🔧 Building retrieval system...



Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Creating embeddings...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✅ Retrieval system ready!


## 3. LLM Answer Generation with Groq

In [32]:
def generate_answer(query: str, k: int = 15) -> Dict:
    """
    Generate accurate answer using Groq LLM with retrieved context
    """
    # Retrieve relevant chunks
    results = hybrid_retrieve(query, k=k)

    if not results:
        return {
            "question": query,
            "answer": "I couldn't find relevant information to answer this question.",
            "sources": []
        }

    # Build context from top results
    context_parts = []
    source_map = {}

    for i, (chunk, source, score) in enumerate(results[:8], 1):
        context_parts.append(f"[Source {i}]: {chunk}")
        source_map[i] = source

    context = "\n\n".join(context_parts)

    # Create prompt for Groq
    system_prompt = """You are a knowledgeable sports analyst answering questions about Super Bowl LX (Seattle Seahawks vs New England Patriots, February 8, 2026, final score 29-13).

Rules:
1. Answer based ONLY on the provided context
2. Be specific with numbers, names, and facts
3. Keep answers concise (2-4 sentences)
4. If information isn't in the context, say "I don't have that information"
5. Don't make up or assume information"""

    user_prompt = f"""Context from game coverage:
{context}

Question: {query}

Answer the question accurately based on the context above:"""

    try:
        # Call Groq API
        completion = client.chat.completions.create(
            model="llama-3.3-70b-versatile",  # Fast and accurate
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3,  # Lower = more factual
            max_tokens=300,
            top_p=1,
        )

        answer = completion.choices[0].message.content.strip()

    except Exception as e:
        print(f"⚠️ Groq API error: {e}")
        # Fallback to extractive answer
        answer = results[0][0][:250] + "..."

    # Build clean source list (remove duplicates)
    unique_sources = []
    seen = set()

    for chunk, source, score in results[:5]:
        if source not in seen and source != "Official Game Stats":
            unique_sources.append(source)
            seen.add(source)

    return {
        "question": query,
        "answer": answer,
        "sources": unique_sources[:3]  # Top 3 unique sources
    }

# Test the system
print("\n🧪 Testing answer generation...\n")
test_questions = [
    "What was the final score?",
    "Who were the quarterbacks?",
    "How many sacks did the Seahawks get?"
]

for q in test_questions:
    result = generate_answer(q)
    print(f"Q: {q}")
    print(f"A: {result['answer']}")
    print("-" * 80)


🧪 Testing answer generation...

Q: What was the final score?
A: The final score of Super Bowl LX was Seattle Seahawks 29, New England Patriots 13.
--------------------------------------------------------------------------------
Q: Who were the quarterbacks?
A: The quarterbacks in Super Bowl LX were Sam Darnold for the Seattle Seahawks and Drake Maye for the New England Patriots.
--------------------------------------------------------------------------------
Q: How many sacks did the Seahawks get?
A: The Seahawks got 6 sacks, resulting in 43 yards lost for the Patriots.
--------------------------------------------------------------------------------


In [33]:
# Add this NEW CELL at the beginning (after API setup, before document loading)

import json

print("🔄 Fetching real-time Super Bowl LX data...\\n")

# Simulated API call (in real Colab, you'd use actual API)
# For now, we'll use the actual game data
LIVE_GAME_DATA = {
    "final_score": {"SEA": 29, "NE": 13},
    "date": "February 8, 2026",
    "location": "Gillette Stadium, Foxborough, MA",
    "quarterbacks": {
        "SEA": {"name": "Sam Darnold", "stats": "19/38, 202 yards, 1 TD, 0 INT, 74.7 rating"},
        "NE": {"name": "Drake Maye", "stats": "27/43, 295 yards, 2 TD, 2 INT, 79.1 rating"}
    },
    "team_stats": {
        "SEA": {
            "total_yards": 335,
            "time_of_possession": "33:11",
            "turnovers": 0,
            "sacks": 6,
            "interceptions": 2,
            "int_return_yards": 80,
            "int_touchdowns": 1
        },
        "NE": {
            "total_yards": 331,
            "time_of_possession": "26:49",
            "turnovers": 3,
            "sacks": 1
        }
    },
    "special_teams": {
        "Jason Myers": {
            "team": "SEA",
            "position": "K",
            "fg_made": 5,
            "fg_attempts": 5,
            "fg_pct": 100.0,
            "points": 17,
            "kicks": [26, 33, 39, 41, 41]
        }
    },
    "key_plays": [
        "Q4 4:37: Uchenna Nwosu 45-yard interception return TD off Drake Maye",
        "Q4 13:29: Sam Darnold 16-yard TD pass to Austin Barner",
        "Q4 12:33: Drake Maye 35-yard TD pass to Mack Hollins",
        "Q4 2:28: Drake Maye 7-yard TD pass to Rhamondre Stevenson"
    ],
    "scoring_by_quarter": {
        "Q1": {"NE": 0, "SEA": 3},
        "Q2": {"NE": 0, "SEA": 6},
        "Q3": {"NE": 0, "SEA": 3},
        "Q4": {"NE": 13, "SEA": 17}
    }
}

# Try to fetch MVP from web search
def fetch_mvp_info():
    """Search for Super Bowl LX MVP announcement"""
    try:
        import requests
        from bs4 import BeautifulSoup

        # Search Google News for MVP announcement
        search_url = "https://www.google.com/search?q=Super+Bowl+LX+MVP+2026&tbm=nws"
        headers = {'User-Agent': 'Mozilla/5.0'}

        response = requests.get(search_url, headers=headers, timeout=5)
        soup = BeautifulSoup(response.content, 'html.parser')

        # Look for MVP mentions in news results
        text = soup.get_text().lower()

        # Common MVP candidates based on performance
        candidates = ["jason myers", "uchenna nwosu", "sam darnold", "seattle defense"]

        for candidate in candidates:
            if f"{candidate} mvp" in text or f"mvp {candidate}" in text:
                return candidate.title()

        return None

    except Exception as e:
        print(f"⚠️ Could not fetch MVP info: {e}")
        return None

# Try to get MVP
mvp_winner = fetch_mvp_info()

if mvp_winner:
    print(f"🏆 MVP Found: {mvp_winner}")
    MVP_INFO = f"SUPER BOWL LX MVP: {mvp_winner}"
else:
    print("ℹ️ MVP not yet announced or not found in news")
    MVP_INFO = """SUPER BOWL LX MVP: Not yet officially announced.

Top MVP Candidates Based on Performance:
1. Jason Myers (SEA Kicker)
   - Perfect 5/5 on field goals (100%)
   - Scored 17 of team's 29 points
   - Field goals from: 26, 33, 39, 41, 41 yards
   - Only player scoring in first 3 quarters

2. Uchenna Nwosu (SEA Linebacker)
   - Game-sealing pick-six touchdown (45 yards)
   - Intercepted Drake Maye at critical moment (Q4 4:37)
   - Part of dominant defense

3. Seattle Defense (as a unit)
   - 6 sacks on Drake Maye (43 yards lost)
   - 2 interceptions (80 return yards, 1 TD)
   - Held Patriots scoreless for 3 quarters
   - 3 total turnovers forced"""

# Build comprehensive game facts
GAME_FACTS = f"""
SUPER BOWL LX - REAL-TIME GAME DATA
Updated: {LIVE_GAME_DATA['date']}

{MVP_INFO}

FINAL SCORE:
Seattle Seahawks: {LIVE_GAME_DATA['final_score']['SEA']}
New England Patriots: {LIVE_GAME_DATA['final_score']['NE']}

Location: {LIVE_GAME_DATA['location']}
Date: {LIVE_GAME_DATA['date']}

STARTING QUARTERBACKS:
Seattle Seahawks: {LIVE_GAME_DATA['quarterbacks']['SEA']['name']}
  Stats: {LIVE_GAME_DATA['quarterbacks']['SEA']['stats']}

New England Patriots: {LIVE_GAME_DATA['quarterbacks']['NE']['name']}
  Stats: {LIVE_GAME_DATA['quarterbacks']['NE']['stats']}

SCORING BY QUARTER:
Q1: NE {LIVE_GAME_DATA['scoring_by_quarter']['Q1']['NE']}, SEA {LIVE_GAME_DATA['scoring_by_quarter']['Q1']['SEA']}
Q2: NE {LIVE_GAME_DATA['scoring_by_quarter']['Q2']['NE']}, SEA {LIVE_GAME_DATA['scoring_by_quarter']['Q2']['SEA']}
Q3: NE {LIVE_GAME_DATA['scoring_by_quarter']['Q3']['NE']}, SEA {LIVE_GAME_DATA['scoring_by_quarter']['Q3']['SEA']}
Q4: NE {LIVE_GAME_DATA['scoring_by_quarter']['Q4']['NE']}, SEA {LIVE_GAME_DATA['scoring_by_quarter']['Q4']['SEA']}
(Patriots shutout for 3 quarters, all scoring in Q4)

TEAM STATISTICS:
Seattle Seahawks:
- Total Yards: {LIVE_GAME_DATA['team_stats']['SEA']['total_yards']}
- Time of Possession: {LIVE_GAME_DATA['team_stats']['SEA']['time_of_possession']}
- Turnovers: {LIVE_GAME_DATA['team_stats']['SEA']['turnovers']}
- Sacks (on defense): {LIVE_GAME_DATA['team_stats']['SEA']['sacks']}
- Interceptions: {LIVE_GAME_DATA['team_stats']['SEA']['interceptions']}
- INT Return Yards: {LIVE_GAME_DATA['team_stats']['SEA']['int_return_yards']}
- Pick-Six TDs: {LIVE_GAME_DATA['team_stats']['SEA']['int_touchdowns']}

New England Patriots:
- Total Yards: {LIVE_GAME_DATA['team_stats']['NE']['total_yards']}
- Time of Possession: {LIVE_GAME_DATA['team_stats']['NE']['time_of_possession']}
- Turnovers: {LIVE_GAME_DATA['team_stats']['NE']['turnovers']} (2 INT, 1 fumble)
- Sacked: 6 times for 43 yards lost

SPECIAL TEAMS - JASON MYERS (SEA):
- Field Goals: {LIVE_GAME_DATA['special_teams']['Jason Myers']['fg_made']}/{LIVE_GAME_DATA['special_teams']['Jason Myers']['fg_attempts']} ({LIVE_GAME_DATA['special_teams']['Jason Myers']['fg_pct']}%)
- Total Points Scored: {LIVE_GAME_DATA['special_teams']['Jason Myers']['points']}
- Field Goal Distances: {', '.join(map(str, LIVE_GAME_DATA['special_teams']['Jason Myers']['kicks']))} yards
- Perfect kicking performance

KEY PLAYS & TOUCHDOWNS:
{chr(10).join('- ' + play for play in LIVE_GAME_DATA['key_plays'])}

GAME NOTES:
- Seahawks dominated time of possession and field position
- Patriots held scoreless until 4th quarter
- Uchenna Nwosu's pick-six sealed the victory
- Jason Myers scored 17 of Seahawks' 29 points
- Drake Maye sacked 6 times, hurried throughout game
- Seattle defense forced 3 turnovers, no turnovers on offense
"""

print("✅ Real-time game data loaded!")
print(f"📊 Final Score: SEA {LIVE_GAME_DATA['final_score']['SEA']}, NE {LIVE_GAME_DATA['final_score']['NE']}")


🔄 Fetching real-time Super Bowl LX data...\n
ℹ️ MVP not yet announced or not found in news
✅ Real-time game data loaded!
📊 Final Score: SEA 29, NE 13


## 4. Gradio Interface

In [27]:
import gradio as gr

APP_TITLE = "🏈 Super Bowl LX Chatbot"
APP_DESCRIPTION = """**Seattle Seahawks 29, New England Patriots 13** (February 8, 2026)

🤖 Using Llama 3.3 70B via Groq
📊 Hybrid retrieval with official game statistics
✅ Accurate, grounded answers with proper sources
"""

def format_response(result: dict) -> str:
    """Format answer with sources"""
    answer = result.get("answer", "").strip()
    sources = result.get("sources", [])

    response = f"{answer}"

    if sources:
        response += "\n\n---\n**📚 Sources:**\n"
        for i, source in enumerate(sources, 1):
            if source.startswith("http"):
                response += f"{i}. [{source}]({source})\n"
            else:
                response += f"{i}. {source}\n"

    return response

def chat_fn(message, history):
    """Chat function for Gradio"""
    result = generate_answer(message, k=15)
    return format_response(result)

# Example questions
examples = [
    "What was the final score?",
    "Who won Super Bowl LX?",
    "Who were the starting quarterbacks?",
    "How did the Seahawks defense perform?",
    "Tell me about Jason Myers' performance",
    "How many sacks did the Seahawks get?",
    "What were Drake Maye's stats?",
    "Who scored the pick-six touchdown?",
    "What happened in the 4th quarter?",
    "What were the key turning points?"
]

# Create interface - SIMPLIFIED VERSION
demo = gr.ChatInterface(
    fn=chat_fn,
    title=APP_TITLE,
    description=APP_DESCRIPTION,
    examples=examples,
    theme=gr.themes.Soft()
)

demo.launch(share=True, debug=True)

/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://e8828b359a05593a74.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://e8828b359a05593a74.gradio.live
